In [7]:
# ------------------------------------------
# IMPORT AND SETUP
# ------------------------------------------

import sys, os
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

#Import the class RawData
from data_import import RawData
raw_data = RawData()

#Import the class RefinedData
from data_handling import RefinedData
ref_data = RefinedData()

#Import the class VisualTable
from utils import VisualTable
pretty_table = VisualTable()

"""
Functions within the class RawData, with arguments:
    get_met(self, weather_station, weather_elements, weather_time, weather_resolution)
    get_nilu(self, threshold, file_path)

Functions within the class RefinedData, with arguments:
    missing_data(self, df)
    show_zeroes(self, df)
"""

'\nFunctions within the class RawData, with arguments:\n    get_met(self, weather_station, weather_elements, weather_time, weather_resolution)\n    get_nilu(self, threshold, file_path)\n\nFunctions within the class RefinedData, with arguments:\n    missing_data(self, df)\n    show_zeroes(self, df)\n'

In [8]:
# ------------------------------------------
# WEATHER DATA - IMPORT AND PROCESSING
# ------------------------------------------

# Variables that can be changed from run to run for the weather data
weather_station = 'SN68860' # The ID of the weather station to fetch data from, can be multiple stations separated by a comma - 'SN68860' is Voll weatherstation
weather_elements = 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)' # The measurements to include in the dataset, here: temperautre, precipitation, and wind speed
weather_time = '2024-01-30/2024-03-19' # The stretch of time to fetch weather data for
weather_resolution = 'P1D' # The level of granularity in the weather data. P1D is once per day. P1H is once per hour.

# Run the module for importing the weather data, and save it to a dataframe
df_weather = raw_data.get_met(weather_station, weather_elements, weather_time, weather_resolution)

# Check if data was fetched successfully and check for missing values
if df_weather is None:
    print("Failed to fetch weather data.")
else:
    missing_data = ref_data.missing_data(df_weather)
    if missing_data is not None:
        print(missing_data)
    else:
        pass

# Displaying the weather data as a table
pretty_table.pretty_data(df_weather)

Data collected from frost.met.no!
There are  245 lines of data in this dataframe.

No missing values found in the data set! 



,value,unit,timeOffset,referenceTime
0,-0.3,degC,PT0H,2024-01-30
1,4.0,degC,PT6H,2024-01-30
2,0.0,mm,PT18H,2024-01-30
3,0.0,mm,PT6H,2024-01-30
4,2.7,m/s,PT0H,2024-01-30
5,2.8,degC,PT0H,2024-01-31
6,-1.5,degC,PT6H,2024-01-31
7,0.0,mm,PT18H,2024-01-31
8,0.0,mm,PT6H,2024-01-31
9,5.7,m/s,PT0H,2024-01-31


In [9]:
# ------------------------------------------
# AIR QUALITY DATA - IMPORT AND PROCESSING
# ------------------------------------------

# Variables that can be changed from run to run for the air quality data
threshold = 95 # The threshold below which we will not use the data for analysis (uptime % of day)
file_path = '../Data/luftkvalitet_trondheim_dag.csv' #Choose the file to get air quality data from

# Run the module for importing the air quality data, and save it to a dataframe
df_quality = raw_data.get_nilu(threshold, file_path)

# Check if data was fetched successfully and check for missing values
if df_quality is None:
    print("Failed to fetch weather data.")
else:
    missing_data = ref_data.missing_data(df_quality)
    if missing_data is not None:
        print("The following datapoints are missing:")
        pretty_table.pretty_data(missing_data)
    else:
        pass

poor_quality_data = ref_data.show_zeroes(df_quality)
print('These datapoints contain no data or data of too poor quality:')
pretty_table.pretty_data(poor_quality_data)

# Displaying the air quality data as a table
pretty_table.pretty_data(df_quality)

Data collected from nilu.com!
There are  367 lines of data in this dataframe.

The following datapoints are missing:


,index,column
0,107,Elgeseter NO µg/m³ Day
1,107,Elgeseter NO2 µg/m³ Day
2,107,Elgeseter NOx µg/m³ Day
3,108,Elgeseter NO µg/m³ Day
4,108,Elgeseter NO2 µg/m³ Day
5,108,Elgeseter NOx µg/m³ Day
6,109,Elgeseter NO µg/m³ Day
7,109,Elgeseter NO2 µg/m³ Day
8,109,Elgeseter NOx µg/m³ Day
9,110,Elgeseter NO µg/m³ Day


These datapoints contain no data or data of too poor quality:


,index,column
0,18,Dekning
1,50,Dekning
2,57,Dekning
3,79,Dekning
4,107,Dekning
5,108,Dekning
6,109,Dekning
7,110,Dekning
8,111,Dekning
9,112,Dekning


,Tid,Elgeseter NO µg/m³ Day,Dekning,Elgeseter NO2 µg/m³ Day,Dekning.1,Elgeseter NOx µg/m³ Day,Dekning.2,Elgeseter PM10 µg/m³ Day,Dekning.3,Elgeseter PM2.5 µg/m³ Day,Dekning.4
0,2024-02-19,21.319116,100.0,47.910158,100.0,80.485768,100.0,11.695174,100.0,9.119713,100.0
1,2024-02-20,36.586944,100.0,48.447275,100.0,104.352126,100.0,20.795971,100.0,14.341274,100.0
2,2024-02-21,28.709966,100.0,43.229089,100.0,87.097916,100.0,11.257472,100.0,5.809501,100.0
3,2024-02-22,8.784671,100.0,16.795990,100.0,30.218967,100.0,4.780046,96.0,0.898309,96.0
4,2024-02-23,4.420641,100.0,11.082582,100.0,17.837321,100.0,1.219465,100.0,0.168885,100.0
5,2024-02-24,1.929630,100.0,7.355437,100.0,10.303912,100.0,2.444950,100.0,0.019941,100.0
6,2024-02-25,5.334206,100.0,17.168342,100.0,25.319009,100.0,15.155823,100.0,2.120134,100.0
7,2024-02-26,5.787199,100.0,22.636717,100.0,31.479558,100.0,6.123895,100.0,2.299517,100.0
8,2024-02-27,21.400212,96.0,30.552252,96.0,63.251776,96.0,18.657540,100.0,3.183995,100.0
9,2024-02-28,11.161719,100.0,27.314969,100.0,44.370076,100.0,33.852309,100.0,4.013046,100.0
